<a href="https://colab.research.google.com/github/ajasja/RosettaCrashCourse/blob/main/notebooks/W01b_scoring_and_minimisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install pyrosetta 
#@markdown Run cells by clicking on the little ▶ play icon to the left of each cell or by going to Runtime->Run all. 

#@markdown Enter the user name and password obtained from https://els2.comotion.uw.edu/product/pyrosetta

#@markdown The notebook must be connected to Google Drive. 

#@markdown Author: Ajasja Ljubetič (ajasja.ljubetic@gmail.com), inspired by https://nbviewer.org/github/RosettaCommons/PyRosetta.notebooks


#only needed for colabfold
!pip install pyrosettacolabsetup py3dmol git+https://github.com/RosettaCommons/pyrosetta_viewer3d.git
import pyrosettacolabsetup; pyrosettacolabsetup.install_pyrosetta()



In [ ]:
#@title Helper functions
def view_single(pose):
  import py3Dmol
  import pyrosetta.distributed.io as io
  view = py3Dmol.view(width=500, height=500)
  view.addModel(io.to_pdbstring(pose),'pdb')
  view.setStyle({'model': 0}, {"cartoon": True})
  view.addStyle({'model': 0}, {"stick": True})
  view.zoomTo()
  view.show()
  return None

def compare_poses(pose1, pose2):
  import py3Dmol
  import pyrosetta.distributed.io as io
  view = py3Dmol.view(width=500, height=500)
  view.addModel(io.to_pdbstring(pose1),'pdb')
  view.addModel(io.to_pdbstring(pose2),'pdb')
  view.setStyle({'model': 0}, {"cartoon": {'color': 'orange'}})
  view.addStyle({'model': 0}, {"stick": {'color': 'orange'}})
  view.setStyle({'model': 1}, {"cartoon": {'color': 'blue'}})
  view.addStyle({'model': 1}, {"stick": {'color': 'blue'}})
  view.zoomTo()
  view.show()
  return None  


# Scoring and minimization
A basic function of Rosetta is calculating the energy or score of a biomolecule. Rosetta has a standard energy function for all-atom calculations as well as several scoring functions for low-resolution protein representations. See https://www.ncbi.nlm.nih.gov/pubmed/28430426 for a review on the all-atom score functions.

You can also tailor an energy function by including scoring terms of your choice with custom weights.

In [ ]:
import pyrosetta; # import the pyrosetta package to access functionality
import pyrosetta.distributed.io as io
pyrosetta.init("-corrections::beta_nov16") # must be called before any other pyrosetta functions. Can accept command line flags

In [ ]:
sfxn = pyrosetta.create_score_function('beta_nov16')
#pritn score function info
print(sfxn)

In [ ]:
#get score function weigths
print(sfxn.weights())

#with line wrapping
print(str(sfxn.weights()).replace(') (',')\n('))
# list of energy terms: https://www.rosettacommons.org/docs/latest/rosetta_basics/scoring/score-types

In [ ]:
# Create a pose from sequence. The structure made is extended and not very realistic
pose = pyrosetta.pose_from_sequence('TESTTHISEPICLIFE')

#score a pose
print(sfxn(pose))
view_single(pose)


# Minimization
The score is very very bad, since side chains are clashing with backbone. It's posible to minimize different degrees of freedom, for example only side chains or only backbone dihedrals. 

Usually minimisation is done in internal coordinates, **cartesian** minimimisation is done in cartesian space (x,y,z coordinates). Which one do you think is faster?

In [ ]:
from pyrosetta.rosetta.core.kinematics import MoveMap
from pyrosetta.rosetta.protocols.minimization_packing import MinMover

scorefxn = pyrosetta.create_score_function('beta_nov16')
#move maps tells minimizer which atoms (degrees of freedom can move)
mm_sidechains = MoveMap()
#Minimise side chains
mm_sidechains.set_chi(True)
#Don't move backbone 
mm_sidechains.set_bb(False)
#create min mover                               #Minimzation type         #tolerance     
minmover_sc = MinMover(mm_sidechains, scorefxn, 'lbfgs_armijo_nonmonotone', 0.0001, True)
#disable cartesian minimisation
minmover_sc.cartesian(False)
#Maximum number of iterations
minmover_sc.max_iter(2000)


In [ ]:
#copy pose
pose_min = pose.clone()
#Mover changes the pose
minmover_sc.apply(pose_min)
print(scorefxn(pose_min))

## Calling XML scripts
Sometimes it is easier to get functunality from Rosetta XML scripts. For example the documnetation is **MUCH** better and there is no need to figure out the name space (eg `pyrosetta.rosetta.protocols.minimization_packing.MinMover`). For example the documentation of all the movers is [here](https://new.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/Movers/Movers-RosettaScripts).

In [ ]:
from rosetta.protocols.rosetta_scripts import XmlObjects
minmover_sc_xml = XmlObjects.static_get_mover('''<MinMover name="min_all_sc" 
        max_iter="2000"
        type="lbfgs_armijo_nonmonotone"
        tolerance="0.0001" cartesian="0"
        chi="1"
        bb="0"  />''')
minmover_sc_xml.score_function(scorefxn)
#copy pose
pose_min_xml = pose.clone()
#Mover changes the pose
minmover_sc_xml.apply(pose_min_xml)
print(scorefxn(pose_min_xml))

## TASK: Print the score of the minimized pose.
What do you see? Also calculate the score per residue by deviding the total score by the number of residues. For stable helical structures the expected `score_per_res` is between -2 and -3.5 units. Is the backbone structure optimal? You can also show the pose using the `view_single` helper function.


In [ ]:
### BEGIN SOLUTION
print(scorefxn(pose_min))
print(scorefxn(pose_min)/pose_min.total_residue())
### END SOLUTION

## TASK: Minimize including the backbone
What do you notice? Is the score lower?  


In [ ]:
### BEGIN SOLUTION
minmover_bb_xml = XmlObjects.static_get_mover('''<MinMover name="min_all_bb" 
        max_iter="2000"
        type="lbfgs_armijo_nonmonotone"
        tolerance="0.0001" cartesian="0"
        chi="1"
        bb="1"  />''')
minmover_bb_xml.score_function(scorefxn)

pose_min_backbone=pose.clone()
minmover_bb_xml.apply(pose_min_backbone)
#view_single(pose_min_backbone)
print(scorefxn(pose_min_backbone))
print(scorefxn(pose_min_backbone)/pose_min_backbone.total_residue())
### END SOLUTION

## TASK: Minimize including the backbone with cartesian minimisation
Cartesian minimization needs a cartesian function! 

What do you notice? Is the score lower?  

In [ ]:
scorefxn_cart =  pyrosetta.create_score_function('beta_nov16_cart')
### BEGIN SOLUTION
pose_min_cart=pose.clone()
minmover_bb_xml.apply(pose_min_cart)
#view_single(pose_min_cart)
print(scorefxn(pose_min_cart))
print(scorefxn_cart(pose_min_cart))
### END SOLUTION

## Bonus task: Set the backbone to helical conformation, minimize side chains and compare the score. 
Minimize also the backbone and sidechains and calculuate the RMSD. Is this structure more stable?

In [ ]:
### BEGIN SOLUTION
from rosetta.core.scoring import CA_rmsd
pose_helix_min = pose.clone()
pose_helix=pose.clone()
for i in range (1, pose_helix.total_residue()+1):
  pose.set_phi(seqpos=i,setting=-60)
  pose.set_psi(seqpos=i,setting=-50)

minmover_bb_xml.apply(pose_helix_min)
print('Score of forced helix pose',scorefxn(pose_helix))
print('Score of bb/sc minimized forced helix pose',scorefxn(pose_helix_min))
print('RMSD',CA_rmsd(pose_helix,pose_helix_min))
print(scorefxn(pose_min))
view_single(pose_helix_min)
### END SOLUTION

# Bonus task: Predict the structe of the peptide using Alphafold (in this notebook). 
Upload the PDB file back to this notebook, minimize the structure and compare the results. Minimize also the backbone and sidechains and calculuate the RMSD. Is this structure more stable?

In [ ]:
### BEGIN SOLUTION
from pyrosetta.io import pose_from_pdb
AF_pose = pose_from_pdb(',PATH TO POSE HERE')
AF_pose_min = AF_pose.clone()
minmover_bb_xml.apply(AF_pose_min)
print('score of minimized AF2 pose',scorefxn(AF_pose_min))
RMSD = CA_rmsd(pose_helix_min,AF_pose_min) 
print ('RMSD',RMSD)
view_single(AF_pose_min)
### END SOLUTION

# Packing
Packing (or repacking) is the process of choosing new rotamers. Behind the scenes it's an annealed Monte-Carlo simulation. Repacking can cross large energy bariers that local minimization methods cannot.




In [ ]:
from pyrosetta.rosetta.core.pack.task import TaskFactory
from pyrosetta.rosetta.core.pack.task import operation
from pyrosetta.rosetta.protocols.minimization_packing import PackRotamersMover 

# Taks factory tells the packer what to do
tf = TaskFactory()
# Do not change amono acid types (so ALA remains ALA)
tf.push_back(operation.RestrictToRepacking())


packer = PackRotamersMover()
packer.task_factory(tf)
packer.score_function(scorefxn)


pose_pack = pose.clone()
packer.apply(pose_pack)

#print what the packer will do
print(packer.task())

print(scorefxn(pose_pack))
view_single(pose_pack).show()

# Quick intro to design
The packer can be used for design -> i.e. changing the amino acid residue types. By default it is set to change any residue into any other residue while minimizing the score

Controlling what the packer does is 80% of using Rosetta! :) 



In [ ]:
# By default the packer does design. 
packer = PackRotamersMover()
packer.score_function(scorefxn)

pose_design = pose.clone()
print(packer.task())


packer.apply(pose_design)
#print what the packer has done
print(packer.task())

print(scorefxn(pose_design))
print(pose.sequence())
print(pose_design.sequence())

view_single(pose_design)

## Task: set the pose into helical shape and redesign the sequence
What do you observe? What is the RMSD if you minimize the sidechains and the backbone+sidechains? 

In [ ]:
### BEGIN SOLUTION
pose_helix_redesign = pose.clone()
for i in range(1, pose_helix_redesign.total_residue()+1):
  pose_helix_redesign.set_phi(i, -60)
  pose_helix_redesign.set_psi(i, -50)


packer = PackRotamersMover()
packer.score_function(scorefxn)
packer.apply(pose_helix_redesign)

pose_helix_redesign_min = pose_helix_redesign.clone()
minmover_bb_xml.apply(pose_helix_redesign_min)

print('Score of redesigned helix: ',scorefxn(pose_helix_redesign), 'Score of minimized redesigned helix: ',scorefxn(pose_helix_redesign_min))
print('RMSD', CA_rmsd(pose_helix_redesign,pose_helix_redesign_min) )
print('Redesigned sequence: ',pose_helix_redesign.sequence(), 'Input sequence', pose.sequence())

view_single(pose_helix_redesign)
view_single(pose_helix_redesign_min)

### END SOLUTION